# Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_server_args`
- `/get_model_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/get_memory_pool_size`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

import requests

server_process = execute_shell_command(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --port=30010
"""
)

wait_for_server("http://localhost:30010")

[2024-11-13 18:32:56] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, is_embedding=False, host='127.0.0.1', port=30010, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=271894015, constrained_json_whitespace_pattern=None, watchdog_timeout=300, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log_interval=40, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, dp_size=1, load_balance_method='round_robin

[2024-11-13 18:33:12 TP0] Init torch distributed begin.


[2024-11-13 18:33:12 TP0] Load weight begin. avail mem=78.59 GB


[2024-11-13 18:33:12 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 11-13 18:33:13 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 11-13 18:33:13 weight_utils.py:288] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.64it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.63it/s]

[2024-11-13 18:33:13 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=75.68 GB
[2024-11-13 18:33:13 TP0] Memory pool end. avail mem=7.42 GB


[2024-11-13 18:33:13 TP0] Capture cuda graph begin. This can take up to several minutes.


[2024-11-13 18:33:21 TP0] max_total_num_tokens=2170757, max_prefill_tokens=16384, max_running_requests=4097, context_len=131072
[2024-11-13 18:33:21] INFO:     Started server process [318930]
[2024-11-13 18:33:21] INFO:     Waiting for application startup.
[2024-11-13 18:33:21] INFO:     Application startup complete.
[2024-11-13 18:33:21] INFO:     Uvicorn running on http://127.0.0.1:30010 (Press CTRL+C to quit)


[2024-11-13 18:33:21] INFO:     127.0.0.1:47060 - "GET /v1/models HTTP/1.1" 200 OK


[2024-11-13 18:33:22] INFO:     127.0.0.1:47072 - "GET /get_model_info HTTP/1.1" 200 OK
[2024-11-13 18:33:22 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-13 18:33:22] INFO:     127.0.0.1:47082 - "POST /generate HTTP/1.1" 200 OK
[2024-11-13 18:33:22] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](../references/sampling_params.md).

In [2]:
url = "http://localhost:30010/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2024-11-13 18:33:26 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 1, cache hit rate: 6.67%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-13 18:33:26 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 6.90, #queue-req: 0
[2024-11-13 18:33:26 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 513.31, #queue-req: 0


[2024-11-13 18:33:27 TP0] Decode batch. #running-req: 1, #token: 121, token usage: 0.00, gen throughput (token/s): 513.25, #queue-req: 0
[2024-11-13 18:33:27] INFO:     127.0.0.1:48544 - "POST /generate HTTP/1.1" 200 OK


## Get Server Args
Get the arguments of a server.

In [3]:
url = "http://localhost:30010/get_server_args"

response = requests.get(url)
print_highlight(response.json())

[2024-11-13 18:33:27] INFO:     127.0.0.1:48556 - "GET /get_server_args HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.

In [4]:
url = "http://localhost:30010/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json["is_generation"] is True
assert response_json.keys() == {"model_path", "is_generation"}

[2024-11-13 18:33:27] INFO:     127.0.0.1:48568 - "GET /get_model_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = "http://localhost:30010/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2024-11-13 18:33:27 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, cache hit rate: 6.25%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-13 18:33:27] INFO:     127.0.0.1:48582 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = "http://localhost:30010/health"

response = requests.get(url)
print_highlight(response.text)

[2024-11-13 18:33:27] INFO:     127.0.0.1:48588 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = "http://localhost:30010/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2024-11-13 18:33:27] INFO:     127.0.0.1:48594 - "POST /flush_cache HTTP/1.1" 200 OK
[2024-11-13 18:33:27 TP0] Cache flushed successfully!


## Get Memory Pool Size

Get the memory pool size in number of tokens.


In [8]:
# get_memory_pool_size

url = "http://localhost:30010/get_memory_pool_size"

response = requests.get(url)
print_highlight(response.text)

[2024-11-13 18:33:27] INFO:     127.0.0.1:48600 - "GET /get_memory_pool_size HTTP/1.1" 200 OK


## Update Weights

Update model weights without restarting the server. Use for continuous evaluation during training. Only applicable for models with the same architecture and parameter size.

In [9]:
# successful update with same architecture and size

url = "http://localhost:30010/update_weights"
data = {"model_path": "meta-llama/Llama-3.2-1B"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."
assert response.json().keys() == {"success", "message"}

[2024-11-13 18:33:27 TP0] Update weights begin. avail mem=4.97 GB


INFO 11-13 18:33:27 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 11-13 18:33:27 weight_utils.py:288] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.92it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.91it/s]

[2024-11-13 18:33:27 TP0] Update weights end.
[2024-11-13 18:33:27 TP0] Cache flushed successfully!
[2024-11-13 18:33:27] INFO:     127.0.0.1:48606 - "POST /update_weights HTTP/1.1" 200 OK


In [10]:
# failed update with different parameter size

url = "http://localhost:30010/update_weights"
data = {"model_path": "meta-llama/Llama-3.2-3B"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to update weights: The size of tensor a (2048) must match "
    "the size of tensor b (3072) at non-singleton dimension 1.\n"
    "Rolling back to original weights."
)

[2024-11-13 18:33:27 TP0] Update weights begin. avail mem=4.97 GB


INFO 11-13 18:33:28 weight_utils.py:243] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 11-13 18:33:28 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 11-13 18:33:28 weight_utils.py:288] No model.safetensors.index.json found in remote.

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]



Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.99it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.99it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]

[2024-11-13 18:33:28 TP0] Failed to update weights: The size of tensor a (2048) must match the size of tensor b (3072) at non-singleton dimension 1.
Rolling back to original weights.
[2024-11-13 18:33:28] INFO:     127.0.0.1:48610 - "POST /update_weights HTTP/1.1" 400 Bad Request


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
terminate_process(server_process)

embedding_process = execute_shell_command(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --port 30020 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server("http://localhost:30020")

[2024-11-13 18:33:40] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, is_embedding=True, host='0.0.0.0', port=30020, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=530988228, constrained_json_whitespace_pattern=None, watchdog_timeout=300, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log_interval=40, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, dp_size=1, load_balance_method='round_robin

[2024-11-13 18:33:56 TP0] Init torch distributed begin.


[2024-11-13 18:33:57 TP0] Load weight begin. avail mem=78.59 GB


[2024-11-13 18:33:57 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 11-13 18:33:57 weight_utils.py:243] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.42s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.64s/it]

[2024-11-13 18:34:09 TP0] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=64.18 GB
[2024-11-13 18:34:09 TP0] Memory pool end. avail mem=7.43 GB


[2024-11-13 18:34:09 TP0] max_total_num_tokens=1025173, max_prefill_tokens=16384, max_running_requests=4005, context_len=131072
[2024-11-13 18:34:09] INFO:     Started server process [319765]
[2024-11-13 18:34:09] INFO:     Waiting for application startup.
[2024-11-13 18:34:09] INFO:     Application startup complete.
[2024-11-13 18:34:09] INFO:     Uvicorn running on http://0.0.0.0:30020 (Press CTRL+C to quit)


[2024-11-13 18:34:09] INFO:     127.0.0.1:59178 - "GET /v1/models HTTP/1.1" 200 OK


[2024-11-13 18:34:10] INFO:     127.0.0.1:59190 - "GET /get_model_info HTTP/1.1" 200 OK
[2024-11-13 18:34:10 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-13 18:34:11] INFO:     127.0.0.1:59200 - "POST /encode HTTP/1.1" 200 OK
[2024-11-13 18:34:11] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = "http://localhost:30020/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2024-11-13 18:34:14 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-13 18:34:14] INFO:     127.0.0.1:59186 - "POST /encode HTTP/1.1" 200 OK


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [13]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process = execute_shell_command(
    """
python -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --port 30030 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server("http://localhost:30030")

[2024-11-13 18:34:24] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, is_embedding=True, host='0.0.0.0', port=30030, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=757462487, constrained_json_whitespace_pattern=None, watchdog_timeout=300, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log_interval=40, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, dp_size=1, load_balanc

[2024-11-13 18:34:40 TP0] Init torch distributed begin.


[2024-11-13 18:34:41 TP0] Load weight begin. avail mem=78.59 GB


[2024-11-13 18:34:41 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 11-13 18:34:41 weight_utils.py:243] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.11it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.13it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.73it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.45it/s]

[2024-11-13 18:34:44 TP0] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=64.48 GB
[2024-11-13 18:34:44 TP0] Memory pool end. avail mem=8.35 GB


[2024-11-13 18:34:45 TP0] max_total_num_tokens=450929, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072
[2024-11-13 18:34:45] INFO:     Started server process [320589]
[2024-11-13 18:34:45] INFO:     Waiting for application startup.
[2024-11-13 18:34:45] INFO:     Application startup complete.
[2024-11-13 18:34:45] INFO:     Uvicorn running on http://0.0.0.0:30030 (Press CTRL+C to quit)


[2024-11-13 18:34:46] INFO:     127.0.0.1:47626 - "GET /get_model_info HTTP/1.1" 200 OK
[2024-11-13 18:34:46 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-13 18:34:46] INFO:     127.0.0.1:47632 - "GET /v1/models HTTP/1.1" 200 OK


[2024-11-13 18:34:46] INFO:     127.0.0.1:47628 - "POST /encode HTTP/1.1" 200 OK
[2024-11-13 18:34:46] The server is fired up and ready to roll!


In [14]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = "http://localhost:30030/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2024-11-13 18:34:57 TP0] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, cache hit rate: 1.38%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-13 18:34:57] INFO:     127.0.0.1:45780 - "POST /classify HTTP/1.1" 200 OK


In [15]:
terminate_process(reward_process)